<a href="https://colab.research.google.com/github/Kotavyshnavi/FMML_LABS_PROJECTS/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [16]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [17]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [18]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [19]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [20]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [21]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [22]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [23]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [24]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [25]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [26]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [27]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [28]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [29]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [30]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

Yes, averaging validation accuracy across multiple splits can provide more consistent results.


_Why averaging helps:_

1. Reduces variability: Averages out fluctuations in validation accuracy due to random splitting.
2. Increases reliability: Provides a more robust estimate of model performance.
3. Mitigates overfitting: Helps identify models that perform well across multiple splits.


_Methods for averaging:_

1. K-Fold Cross-Validation (CV): Divide data into k folds, train on k-1 folds, evaluate on remaining fold.
2. Stratified K-Fold CV: Maintains class balance in each fold.
3. Repeated K-Fold CV: Repeats k-fold CV multiple times with different random splits.


_Benefits:_

1. Improved estimation of model performance
2. Better comparison of models
3. Increased confidence in results
4. Reduced risk of overestimation or underestimation


_Considerations:_

1. Computational cost: Increases with number of splits and repeats.
2. Choice of k: Balances bias-variance tradeoff (e.g., k=5 or k=10).
3. Random seed: Ensures reproducibility.


_Best practices:_

1. Use Stratified K-Fold CV for imbalanced datasets.
2. Choose suitable k and repeat values.
3. Monitor standard deviation or variance of validation accuracy.
4. Visualize results with error bars or box plots.


_Python libraries:_

1. Scikit-learn (sklearn.model_selection.KFold, sklearn.model_selection.StratifiedKFold)
2. TensorFlow (tf.keras.utils.KFold)
3. PyTorch (torch.utils.data.DataLoader with batch_sampler)


By averaging validation accuracy across multiple splits, you can increase the reliability and robustness of your model evaluation.


Do you have any specific questions about implementing k-fold cross-validation?

Averaging validation accuracy across multiple splits can provide a more accurate estimate of test accuracy, but it's not a guarantee.


_Why averaging improves estimation:_

1. Reduces variability: Averages out fluctuations in validation accuracy due to random splitting.
2. Increases reliability: Provides a more robust estimate of model performance.
3. Better represents population: Averages over multiple splits can better approximate the true population performance.


_Conditions for improved estimation:_

1. Large enough sample size
2. Representative validation sets
3. Sufficient number of splits (k)
4. Independent and identically distributed (i.i.d.) data


_Limitations:_

1. Biased validation sets: If validation sets are not representative, averaging won't improve estimation.
2. Overfitting to validation sets: Models may still overfit to validation sets, especially with small datasets.
3. Different validation and test distributions: Averaging validation accuracy may not reflect test accuracy if distributions differ.


_When averaging is most effective:_

1. Large datasets with minimal overfitting
2. Simple models with minimal capacity for overfitting
3. Well-regularized models


_Best practices:_

1. Use stratified sampling to maintain class balance.
2. Ensure validation sets are representative.
3. Monitor test accuracy for final evaluation.
4. Consider using techniques like bootstrapping or Monte Carlo cross-validation.


_Remember:_

1. Validation accuracy is an estimate of test accuracy.
2. Averaging validation accuracy improves estimation, but not always.
3. Test accuracy remains the gold standard for evaluation.


To further improve estimation:


1. Use techniques like cross-validation with test sets.
2. Employ Bayesian methods for uncertainty estimation.
3. Consider using probabilistic models.


Do you have any questions about implementing cross-validation or estimating test accuracy?

The number of iterations (or repeats) in cross-validation significantly impacts the estimate's quality.


_Effects of increasing iterations:_

*Positive effects:*

1. Reduced variability: More iterations decrease the standard error of the estimate.
2. Improved precision: Estimates become more precise and reliable.
3. Better representation: More iterations capture the data's underlying patterns.


*Negative effects:*

1. Computational cost: Increased iterations raise computational time and resources.
2. Overfitting risk: Excessive iterations may lead to overfitting, especially with complex models.


*Optimal number of iterations:*

1. Depends on dataset size, complexity, and model.
2. Typically, 5-10 iterations (k-folds) are sufficient.
3. More iterations (e.g., 20-50) may be needed for small datasets or noisy data.


*Convergence:*

1. Estimate convergence: Monitor the estimate's stability across iterations.
2. Stop when convergence is reached: No significant improvement with additional iterations.


*Guidelines:*

1. Start with 5-10 iterations and assess convergence.
2. Increase iterations if estimate variability is high.
3. Balance computational cost and precision.


*Theoretical background:*

1. Law of Large Numbers (LLN): Averages converge to expected values with increasing iterations.
2. Central Limit Theorem (CLT): Estimates approach normal distribution with increasing iterations.


*Python libraries:*

1. Scikit-learn: `KFold`, `StratifiedKFold` with `n_splits` parameter.
2. TensorFlow: `tf.keras.utils.KFold` with `n_splits` parameter.


*Example code:*
```
from sklearn.model_selection import KFold

# Define dataset and model
X, y = ...

# Create KFold object with 10 iterations
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
scores = []
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # Train model and evaluate on validation set
    model.fit(X_train, y_train)
    scores.append(model.score(X_val, y_val))

# Calculate average score
avg_score = np.mean(scores)
```
By carefully selecting the number of iterations, you can achieve a better estimate of your model's performance.


Do you have any questions about implementing cross-validation or estimating model performance?

Increasing iterations can help alleviate issues with small train or validation datasets, but there are limitations.


*Benefits of increasing iterations for small datasets:*

1. Reduced variability: More iterations decrease the standard error of the estimate.
2. Improved precision: Estimates become more precise and reliable.
3. Better representation: More iterations capture the data's underlying patterns.


*Limitations:*

1. Overfitting risk: Excessive iterations may lead to overfitting, especially with complex models.
2. Computational cost: Increased iterations raise computational time and resources.
3. Data exhaustion: Repeatedly sampling from a small dataset may lead to similar training and validation sets.


*Alternatives to increasing iterations:*

1. Data augmentation: Artificially increase dataset size through transformations.
2. Transfer learning: Leverage pre-trained models and fine-tune on small dataset.
3. Regularization techniques: Prevent overfitting with methods like dropout, L1/L2 regularization.
4. Bayesian methods: Incorporate prior knowledge and uncertainty estimation.
5. Ensemble methods: Combine multiple models to improve performance.


*Minimum dataset size requirements:*

1. Simple models (e.g., linear regression): 10-50 samples
2. Moderate models (e.g., decision trees): 50-200 samples
3. Complex models (e.g., neural networks): 200-1000 samples


*Guidelines for small datasets:*

1. Start with simple models and incremental complexity.
2. Use regularization techniques and early stopping.
3. Employ data augmentation and transfer learning.
4. Monitor performance on validation set.


*Python libraries for small dataset handling:*

1. Scikit-learn: `KFold`, `StratifiedKFold` with `n_splits` parameter
2. TensorFlow: `tf.keras.utils.KFold` with `n_splits` parameter
3. PyTorch: `torch.utils.data.DataLoader` with `batch_sampler`


*Example code for data augmentation:*
```
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define dataset
X, y = ...

# Create data generator with augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=30,
    zoom_range=[0.8, 1.2]
)

# Train model with augmented data
model.fit(datagen.flow(X, y, batch_size=32),
          validation_data=(X_val, y_val),
          epochs=10)
```
By understanding the limitations and alternatives, you can effectively handle small train or validation datasets.


Do you have any questions about implementing data augmentation or regularization techniques?